In [1]:
#importing libraries
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import os
import keras

In [2]:
# There are two classes of interests among the images in dataset
cls_of_int = ['Covid', 'Normal']

In [3]:
#setting the path variables for train, test and validation data
basePath = "Data"
trainPath = os.path.join(basePath,"train")
testPath = os.path.join(basePath,"test")
validPath = os.path.join(basePath,"valid")

trainCovid = os.path.join(trainPath,"Covid")
trainNormal = os.path.join(trainPath,"Normal")

testCovid = os.path.join(testPath,"Covid")
testNormal = os.path.join(testPath,"Normal")

validCovid = os.path.join(basePath,"valid","Covid")
validNormal = os.path.join(basePath,"valid","Normal")

In [4]:
# using a data generator for data augmentation, the volume of training data is increased using data augmentation techniques
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip = False,
    fill_mode='nearest',
    featurewise_center = True,
    featurewise_std_normalization = True)

In [5]:
# The data generator for validation data, normalizing of the image is done
valid_data_generator = ImageDataGenerator(rescale = 1./255)

In [6]:
# The data generator for test data, normalizing of the image is done
test_data_generator = ImageDataGenerator(rescale = 1./255)

In [7]:
# Read data from directory for training data
train_generator = train_data_generator.flow_from_directory(
    directory=trainPath,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42
)

Found 11047 images belonging to 2 classes.


In [8]:
# Read data from directory for validation data
valid_generator = valid_data_generator.flow_from_directory(
    directory= validPath,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42
)

Found 2208 images belonging to 2 classes.


In [9]:
# Read data from directory for testing data
test_generator = test_data_generator.flow_from_directory(
    directory= testPath,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=11,
    class_mode='binary',
    shuffle=False,
    seed=42
)

Found 2761 images belonging to 2 classes.


In [10]:
# Downloading the MobileNetV3 model.
pretrained_model = tf.keras.applications.MobileNetV3Large(
        weights='imagenet',
        include_top=False ,
        input_shape=(224,224,3)
    )
pretrained_model.trainable = False

# Added Pooling layer and a sigmoid activation layer.     
model = tf.keras.Sequential([ 
        pretrained_model,   
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

17620992/17605208 [==============================] - 10s 1us/step


In [11]:
# Compiling the model with Adam optimizer and binary cross entropy loss function
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = opt ,
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [12]:
# To display the summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
MobilenetV3large (Functional (None, 1, 1, 1280)        4226432   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 4,228,994
Trainable params: 2,562
Non-trainable params: 4,226,432
_________________________________________________________________


In [13]:
# specifying the batch sizes
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [14]:
# Fitting the model with 20 epochs
model.fit_generator(generator = train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)

D:\Users\Ajay\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
D:\Users\Ajay\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
D:\Users\Ajay\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20
345/345 [==============================] - 235s 649ms/step - loss: 0.5714 - accuracy: 0.7370 - val_loss: 0.5618 - val_accuracy: 0.7382
Epoch 2/20
345/345 [==============================] - 133s 384ms/step - loss: 0.5547 - accuracy: 0.7378 - val_loss: 0.5482 - val_accuracy: 0.7382
Epoch 3/20
345/345 [==============================] - 141s 409ms/step - loss: 0.5448 - accuracy: 0.7387 - val_loss: 0.5392 - val_accuracy: 0.7382
Epoch 4/20
345/345 [==============================] - 128s 371ms/step - loss: 0.5397 - accuracy: 0.7389 - val_loss: 0.5329 - val_accuracy: 0.7382
Epoch 5/20
345/345 [==============================] - 127s 367ms/step - loss: 0.5384 - accuracy: 0.7399 - val_loss: 0.5309 - val_accuracy: 0.7382
Epoch 6/20
345/345 [==============================] - 130s 378ms/step - loss: 0.5359 - accuracy: 0.7394 - val_loss: 0.5284 - val_accuracy: 0.7405
Epoch 7/20
345/345 [==============================] - 137s 396ms/step - loss: 0.5282 - accuracy: 0.7409 - val_loss: 0.5293 -

In [17]:
score = model.evaluate(valid_generator,batch_size=32)
print("Validation Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

69/69 [==============================] - 5s 77ms/step - loss: 0.5284 - accuracy: 0.7382
Accuracy: 73.82%
Loss:  0.5283759832382202


In [18]:
score = model.evaluate(test_generator,batch_size=11)
print("Test Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

251/251 [==============================] - 8s 31ms/step - loss: 0.5237 - accuracy: 0.7392
Accuracy: 73.92%
Loss:  0.5237259864807129


In [19]:
model.save('mobilev3large')

INFO:tensorflow:Assets written to: mobilev3large\assets


INFO:tensorflow:Assets written to: mobilev3large\assets
D:\Users\Ajay\anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [20]:
loaded_model = keras.models.load_model('mobilev3large')

In [21]:
# Calculating the validation score
valscore = loaded_model.evaluate(valid_generator)
print("Accuracy: {:.2f}%".format(valscore[1] * 100))

69/69 [==============================] - 7s 81ms/step - loss: 0.5284 - accuracy: 0.7382
Accuracy: 73.82%


In [22]:
# Calculating the test score
score = loaded_model.evaluate(test_generator)
print("Test Accuracy on preloaded: {:.2f}%".format(score[1] * 100))

251/251 [==============================] - 8s 31ms/step - loss: 0.5237 - accuracy: 0.7392
Accuracy: 73.92%
